In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://dlcdn.apache.org/spark/spark-3.4.4/spark-3.4.4-bin-hadoop3.tgz
!tar -xzf spark-3.4.4-bin-hadoop3.tgz
!pip install -q pyspark==3.4.4

--2025-01-21 20:12:44--  https://dlcdn.apache.org/spark/spark-3.4.4/spark-3.4.4-bin-hadoop3.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 388988563 (371M) [application/x-gzip]
Saving to: ‘spark-3.4.4-bin-hadoop3.tgz’

spark-3.4.4-bin-had 100%[===================>] 370.97M   188MB/s    in 2.0s    

2025-01-21 20:12:48 (188 MB/s) - ‘spark-3.4.4-bin-hadoop3.tgz’ saved [388988563/388988563]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.4/311.4 MB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 12.2 MB/s eta 0:00:00


In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.4-bin-hadoop3"


In [7]:
!pwd

/content


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Etapa 1

In [13]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SQLContext

spark = SparkSession \
        .builder \
        .master ("local[*]") \
        .appName("Exercicio Intro") \
        .getOrCreate()

path = '/content/drive/MyDrive/Colab Notebooks/txt/nomes_aleatorios.txt'

df_nomes = spark.read.csv(path)

df_nomes.show(5)


+----------------+
|             _c0|
+----------------+
|  Frances Bennet|
|   Jamie Russell|
|  Edward Kistler|
|   Sheila Maurer|
|Donald Golightly|
+----------------+
only showing top 5 rows



# Etapa 2

In [16]:
# Imprimir o schema do dataframe
df_nomes.printSchema()

root
 |-- _c0: string (nullable = true)



In [17]:
# Renomear a coluna do dataframe para nome
df_nomes = df_nomes.withColumnRenamed("_c0", "Nomes")

df_nomes.show(10)
df_nomes.printSchema()

+-----------------+
|            Nomes|
+-----------------+
|   Frances Bennet|
|    Jamie Russell|
|   Edward Kistler|
|    Sheila Maurer|
| Donald Golightly|
|       David Gray|
|      Joy Bennett|
|      Paul Kriese|
|Berniece Ornellas|
|    Brian Farrell|
+-----------------+
only showing top 10 rows

root
 |-- Nomes: string (nullable = true)



# Etapa 3

In [18]:
# Importa bibliotecas random e when, o random para gerar aleatoriedade da condicao e o when que é a condicional

from pyspark.sql.functions import rand, when

# Adicionar a coluna Escolaridade com valores aleatórios entre Fundamental (60%), Médio(30%) e Superior(10%)

df_nomes = df_nomes.withColumn(
    "Escolaridade",
    when(rand() < 0.60, "Fundamental")
    .when(rand() < 0.90, "Médio")
    .otherwise("Superior"))

df_nomes.show(10)
df_nomes.printSchema()

+-----------------+------------+
|            Nomes|Escolaridade|
+-----------------+------------+
|   Frances Bennet| Fundamental|
|    Jamie Russell| Fundamental|
|   Edward Kistler|       Médio|
|    Sheila Maurer| Fundamental|
| Donald Golightly| Fundamental|
|       David Gray|       Médio|
|      Joy Bennett|       Médio|
|      Paul Kriese| Fundamental|
|Berniece Ornellas|       Médio|
|    Brian Farrell| Fundamental|
+-----------------+------------+
only showing top 10 rows

root
 |-- Nomes: string (nullable = true)
 |-- Escolaridade: string (nullable = false)



# Etapa 4

In [45]:
# Cria a coluna Pais e coloca paises da America do Sul com probabilidade 1/13

df_nomes = df_nomes.withColumn(
    "Pais",
    when(rand() < 1/13, "Argentina")
    .when(rand() < 2/13, "Bolivia")
    .when(rand() < 3/13, "Brasil")
    .when(rand() < 4/13, "Chile")
    .when(rand() < 5/13, "Colombia")
    .when(rand() < 6/13, "Equador")
    .when(rand() < 7/13, "Guiana")
    .when(rand() < 8/13, "Paraguai")
    .when(rand() < 9/13, "Peru")
    .when(rand() < 10/13, "Suriname")
    .when(rand() < 11/13, "Uruguai")
    .when(rand() < 12/13, "Venezuela")
    .otherwise("Guiana Francesa")
)

df_nomes.show(10)
df_nomes.printSchema()

+-----------------+------------+--------+
|            Nomes|Escolaridade|    Pais|
+-----------------+------------+--------+
|   Frances Bennet| Fundamental|   Chile|
|    Jamie Russell| Fundamental|  Brasil|
|   Edward Kistler|       Médio|  Brasil|
|    Sheila Maurer| Fundamental| Bolivia|
| Donald Golightly| Fundamental|Colombia|
|       David Gray|       Médio|  Brasil|
|      Joy Bennett|       Médio|  Brasil|
|      Paul Kriese| Fundamental| Bolivia|
|Berniece Ornellas|       Médio|  Guiana|
|    Brian Farrell| Fundamental|   Chile|
+-----------------+------------+--------+
only showing top 10 rows

root
 |-- Nomes: string (nullable = true)
 |-- Escolaridade: string (nullable = false)
 |-- Pais: string (nullable = false)



# Etapa 5

In [46]:
# Importa a biblioteca floor que é para arredondamento

from pyspark.sql.functions import floor

# Cria uma nova coluna AnoNascimento que os valores vao variar entre 1945 a 2010

df_nomes = df_nomes.withColumn("AnoNascimento", floor(rand() * 65 + 1945).cast("int"))

df_nomes.show(10)
df_nomes.printSchema()

+-----------------+------------+--------+-------------+
|            Nomes|Escolaridade|    Pais|AnoNascimento|
+-----------------+------------+--------+-------------+
|   Frances Bennet| Fundamental|   Chile|         2007|
|    Jamie Russell| Fundamental|  Brasil|         1995|
|   Edward Kistler|       Médio|  Brasil|         1996|
|    Sheila Maurer| Fundamental| Bolivia|         1997|
| Donald Golightly| Fundamental|Colombia|         1965|
|       David Gray|       Médio|  Brasil|         1975|
|      Joy Bennett|       Médio|  Brasil|         1997|
|      Paul Kriese| Fundamental| Bolivia|         2001|
|Berniece Ornellas|       Médio|  Guiana|         1954|
|    Brian Farrell| Fundamental|   Chile|         1954|
+-----------------+------------+--------+-------------+
only showing top 10 rows

root
 |-- Nomes: string (nullable = true)
 |-- Escolaridade: string (nullable = false)
 |-- Pais: string (nullable = false)
 |-- AnoNascimento: integer (nullable = true)



# Etapa 6

In [47]:
# Importa a biblioteca col que ajuda a manipulacao das colunas

from pyspark.sql.functions import col

# Cria um novo dataframe e filtra os valores do AnoNascimento para maior ou igual a 2000

df_select = df_nomes.select("Nomes",
                            "Escolaridade",
                            "Pais",
                            col("AnoNascimento").cast("int").alias("AnoNascimento")) \
                    .filter(col("AnoNascimento") >= 2000)

df_select.show(10)

+----------------+------------+--------+-------------+
|           Nomes|Escolaridade|    Pais|AnoNascimento|
+----------------+------------+--------+-------------+
|  Frances Bennet| Fundamental|   Chile|         2007|
|     Paul Kriese| Fundamental| Bolivia|         2001|
|    Charles Hill| Fundamental|  Guiana|         2005|
|    Rebecca Snow| Fundamental| Equador|         2009|
|Wallace Mitchell|       Médio|Paraguai|         2007|
|  Katrina Graham|       Médio|Colombia|         2000|
| Jerry Chynoweth| Fundamental|  Brasil|         2002|
|   Michael Agnew| Fundamental|   Chile|         2008|
|Flossie Anderson| Fundamental|  Brasil|         2001|
|      Ned Tester| Fundamental|   Chile|         2003|
+----------------+------------+--------+-------------+
only showing top 10 rows



# Etapa 7

In [48]:
# Cria uma view chamada pessoas

df_nomes.createOrReplaceTempView ("pessoas")

# Utiliza a funcao spark.sql que vai utilizar as funcoes do SQL que nesse caso é o mesmo da Etapa 6

spark.sql("""
    SELECT
        Nomes,
        Escolaridade,
        Pais,
        AnoNascimento
    FROM pessoas
    WHERE AnoNascimento >= 2000
""").show()

+-----------------+------------+--------+-------------+
|            Nomes|Escolaridade|    Pais|AnoNascimento|
+-----------------+------------+--------+-------------+
|   Frances Bennet| Fundamental|   Chile|         2007|
|      Paul Kriese| Fundamental| Bolivia|         2001|
|     Charles Hill| Fundamental|  Guiana|         2005|
|     Rebecca Snow| Fundamental| Equador|         2009|
| Wallace Mitchell|       Médio|Paraguai|         2007|
|   Katrina Graham|       Médio|Colombia|         2000|
|  Jerry Chynoweth| Fundamental|  Brasil|         2002|
|    Michael Agnew| Fundamental|   Chile|         2008|
| Flossie Anderson| Fundamental|  Brasil|         2001|
|       Ned Tester| Fundamental|   Chile|         2003|
|      Milton Rowe| Fundamental|Paraguai|         2002|
|     Jerry Remick|       Médio|Colombia|         2006|
|   Kristyn Phelps|       Médio|  Brasil|         2004|
|     Ricky Scheel| Fundamental| Equador|         2000|
|Anthony Hernandez|       Médio| Equador|       

# Etapa 8

In [49]:
# Um novo dataframe que vai contar o numero de pessoas entre 1980 a 1994

df_millennials = df_nomes.select("Nomes",
                            "Escolaridade",
                            "Pais",
                            col("AnoNascimento").cast("int").alias("AnoNascimento")) \
                    .filter(col("AnoNascimento").between(1980, 1994))

linhas = df_millennials.count()
print(linhas)

2306369


# Etapa 9

In [50]:
# Cria uma nova view chamada pessoas_2

df_nomes.createOrReplaceTempView ("pessoas_2")

# Conta a quantidade de pessoas de 1980 a 1994 que deve ser a mesma que a etapa 8

df_millennials = spark.sql("""
    SELECT COUNT(*) AS Total
    FROM pessoas
    WHERE AnoNascimento >= 1980
    AND AnoNascimento <= 1994
""")

df_millennials.show()

+-------+
|  Total|
+-------+
|2306369|
+-------+



# Etapa 10

In [56]:
# Cria uma view pessoas_3

df_nomes.createOrReplaceTempView("pessoas_3")

# Cria um novo dataframe utilizando a funcao spark.sql que vai criar uma nova coluna chamada Geracao que vai distribuir em valores diferentes de geracoes

df_final = spark.sql("""
    SELECT
        CASE
            WHEN AnoNascimento >= 1944 AND AnoNascimento <= 1964 THEN 'Baby Boomers'
            WHEN AnoNascimento >= 1965 AND AnoNascimento <= 1979 THEN 'Geracao X'
            WHEN AnoNascimento >= 1980 AND AnoNascimento <= 1994 THEN 'Millennials (Geracao Y)'
            WHEN AnoNascimento >= 1995 AND AnoNascimento <= 2015 THEN 'Geracao Z'
        END AS Geracao,
        Pais,
        COUNT(*) AS Quantidade
    FROM pessoas_3
    GROUP BY
        CASE
            WHEN AnoNascimento >= 1944 AND AnoNascimento <= 1964 THEN 'Baby Boomers'
            WHEN AnoNascimento >= 1965 AND AnoNascimento <= 1979 THEN 'Geracao X'
            WHEN AnoNascimento >= 1980 AND AnoNascimento <= 1994 THEN 'Millennials (Geracao Y)'
            WHEN AnoNascimento >= 1995 AND AnoNascimento <= 2015 THEN 'Geracao Z'
        END,
        Pais
    ORDER BY Pais, Geracao, Quantidade DESC
""")

df_final.show()

+--------------------+---------+----------+
|             Geracao|     Pais|Quantidade|
+--------------------+---------+----------+
|        Baby Boomers|Argentina|    236545|
|           Geracao X|Argentina|    177533|
|           Geracao Z|Argentina|    177553|
|Millennials (Gera...|Argentina|    178125|
|        Baby Boomers|  Bolivia|    437998|
|           Geracao X|  Bolivia|    328187|
|           Geracao Z|  Bolivia|    328477|
|Millennials (Gera...|  Bolivia|    326072|
|        Baby Boomers|   Brasil|    556242|
|           Geracao X|   Brasil|    415013|
|           Geracao Z|   Brasil|    417022|
|Millennials (Gera...|   Brasil|    416176|
|        Baby Boomers|    Chile|    568011|
|           Geracao X|    Chile|    426063|
|           Geracao Z|    Chile|    426729|
|Millennials (Gera...|    Chile|    426343|
|        Baby Boomers| Colombia|    492470|
|           Geracao X| Colombia|    368375|
|           Geracao Z| Colombia|    368764|
|Millennials (Gera...| Colombia|